### CS 210: Data Management for Data Science
#### Fall 2023

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

## Homework 3: <font color="blue">Movie Recommendations</font>

### Overview
In this assignment, you'll implement a prototype movie
recommendation program by filling in the function implementations in
the given `hw3.py` template file.

**Make sure you implement your functions in `hw3.py`**. Do not implement
the functions in this notebook. They will be imported here from `hw3.py`, 
so you can test them directly. 

You can run your tests on the given input files, but you're
encouraged to make your own test files as well, to make sure that
you cover the various paths of logic in your functions. You don't
have to submit any of your test files.

You may assume that all parameter values given to your functions
will be valid. Also, in any function that requires the returned
values to be sorted or ranked, ties may be broken arbitrarily
between equal values.

You may assume that input files will be correctly formatted, and
data types will be as expected.

For all rating computations, do not round up (or otherwise modify)
the rating unless otherwise specified.

#### Input Files

**Ratings file**: A text file that contains movie ratings. Each line has the id of the user who rated the movie, the id of the movie, and its rating (range 0-5 inclusive). A movie can have multiple ratings from different users. A user can rate multiple movies but can rate a particular movie only once. 

**Movies file**: A text file that contains movies and their genres. Each line has a movie id, the name of the movie, the year it was released, and the genre of the movie (e.g, Action, Drama, etc.). 

There are sample movies and ratings files provided on Codebench. 

### Part 1: Reading Data

In [ ]:
import hw3

In [ ]:
from hw3 import read_movies_data, read_ratings_data

#### 1.1 (8 points) Read Movies File

Write a function `read_movies_data(f)` that takes in a movies file and returns a pandas DataFrame, where the index is the movie id and the columns are "title", "year", and "genre" (in that order). 


In [7]:
import pandas as pd

def read_movies_data(movies_file):
    # Define the column names for the DataFrame
    column_names = ["movie_id", "title", "year", "genre"]
    
    movies_df = pd.read_csv(movies_file, names=column_names, sep='\t')
    
    movies_df.set_index('movie_id', inplace=True)
    
    return movies_df

movies_df = read_movies_data("moviesSample.txt")



In [8]:
assert movies_df.shape == (9, 3)
assert movies_df.iloc[2, 2] == "Comedy"
assert movies_df.loc[6, "title"] == "Heat"
assert all(movies_df.columns == ["title", "year", "genre"])
assert list(movies_df.index) == [1, 2, 3, 4, 5, 6, 8, 9, 10]

AssertionError: 

#### 1.2 (8 points) Read Ratings File
Write a function `read_ratings_data(f)` that takes in a ratings
file name and returns a dictionary. The dictionary should have
movie_ratings = read_ratings_data("ratingsSample.csv")
the movie id as key, and the corresponding list of ratings as value.

In [18]:
import csv

def read_ratings_data(f):
    movie_ratings = {}  
    
    with open(f, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  
        for row in reader:
            user_id, movie_id, rating = int(row[0]), int(row[1]), float(row[2])
            
            if movie_id in movie_ratings:
                movie_ratings[movie_id].append(rating)
            else:
                movie_ratings[movie_id] = [rating]
    
    return movie_ratings

movie_ratings = read_ratings_data("ratingsSample.csv")




In [19]:
assert sum(movie_ratings[5]) == 24

### Part 2: Processing Data

In [28]:
from hw3 import get_movie, create_genre_dict, calculate_average_rating

#### 2.1 (8 points) Genre Dictionary

Write a function `create_genre_dict(df)` which takes a movies DataFrame as a parameter and returns a dictionary with each genre as a key and a list of movie ids as values. 

Example: 
```
{
    "Adventure": [1, 2, 8],
    "Comedy": [3, 4, 5],
    "Action": [6, 9, 10]
}
```

In [21]:
import pandas as pd

def create_genre_dict(movies_df):
    genre_dict = {}


    for index, row in movies_df.iterrows():
        genres = row["genre"]

        if isinstance(genres, str):

            if '|' in genres:
                genres = genres.split('|')
                for genre in genres:
                    if genre in genre_dict:
                        genre_dict[genre].append(index)
                    else:
                        genre_dict[genre] = [index]
            else:
                if genres in genre_dict:
                    genre_dict[genres].append(index)
                else:
                    genre_dict[genres] = [index]

    return genre_dict


#### 2.2 (8 points) Average Rating

Write a function `calculate_average_rating` which takes as input a ratings dictionary like that created in Part 1.2 and a movies DataFrame of the kind created in Part 1.1. The function should return a pandas `Series` where the index corresponds to the movie id and the value is the average of all the ratings for the movie. 

In [25]:
import pandas as pd

def calculate_average_rating(ratings_dict, movies_df):
    avg_ratings = {}
    
    for movie_id, ratings in ratings_dict.items():
        avg_rating = sum(ratings) / len(ratings)
        avg_ratings[movie_id] = avg_rating
    
    avg_ratings_series = pd.Series(avg_ratings)
    
    return avg_ratings_series


avg_ratings = calculate_average_rating(movie_ratings, movies_df)


assert avg_ratings.loc[3] == 4.0


In [26]:
assert avg_ratings.loc[3] == 4.0

### Part 3: Recommendations

In [27]:
from hw3 import get_popular_movies, filter_movies, get_popular_in_genre
from hw3 import get_genre_rating, get_movie_of_the_year

#### 3.1 (8 points) Popularity Based

In services such as Netflix and Spotify, you often see
recommendations with the heading "Popular movies" or "Trending top
10".

Write a function `get_popular_movies` that takes as parameters a pandas Series
of movie-to-average rating (as created in Part 2.2) and an integer $n$ 
(default should be 10). 
The function should return a
pandas Series (movie_id:average rating, same structure as input) of top $n$ movies based on the average ratings. If there
are fewer than $n$ movies, it should return all movies in order of top
average ratings. The Series should be sorted in descending order by average rating. 
```
[In]: get_popular_movies(avg_ratings, 4)
[Out]: 
6    4.250000
5    4.000000
3    4.000000
1    3.833333
dtype: float64
```

In [94]:
import pandas as pd

def get_popular_movies(avg_ratings, n=10):
    sorted_movies = avg_ratings.sort_values(ascending=False)
    return sorted_movies.head(n)

pop_movies = get_popular_movies(avg_ratings, 4)
print(pop_movies)

6    4.25
3    4.00
5    4.00
1    3.80
dtype: float64


In [95]:
pop_movies = get_popular_movies(avg_ratings, 4)
assert pop_movies.is_monotonic_decreasing
assert set(pop_movies.index) == {6, 5, 3, 1}

#### 3.2 (8 point) Threshold Rating

Write a function `filter_movies` that takes as parameters a pandas Series of
movie-to-average rating (same as for the popularity based
function above), and a threshold rating with default value of 3. The
function should filter movies based on the threshold rating, and
return a dictionary with same structure as the input. For example,
if the threshold rating is 3.5, the returned dictionary should have
only those movies from the input whose average rating is equal to or
greater than 3.5.

Example: 
```
[In]: filter_movies(avg_ratings, 3.2)

[Out]: 
1    3.833333
2    3.416667
3    4.000000
5    4.000000
6    4.250000
9    3.333333
dtype: float64
```

In [96]:
import pandas as pd

def filter_movies(avg_ratings, threshold=3.0):
    filtered_movies = avg_ratings[avg_ratings >= threshold]
    
    return filtered_movies

In [97]:
good_movies = filter_movies(avg_ratings, 3.2)
assert good_movies.shape[0] == 6
assert all(good_movies.values >= 3.2)

#### 3.3 (8 points) Popularity & genre based

In most recommendation systems, genre of the movie/song/book plays
an important role. Often features like popularity, genre, artist are
combined to present recommendations to a user.

Write a function `get_popular_in_genre` which takes as parameters a genre, a genre-to-movies dictionary (as created in 2.1), a pandas Series of movie-to-average rating (as created in 2.2) and an integer $n$ (default 5), and returns the top $n$ movies for the given genre. The return should be a Series of movie-to-average rating that make the cut, i.e., the same format as the input. If there are fewer than $n$ movies, all movies for the genre are returned, in decreasing order by average rating. 
```
[In]: get_popular_in_genre("Adventure", genre_dict, avg_ratings, 2)
[Out]: 
1    3.833333
2    3.416667
dtype: float64
```

In [98]:
def get_popular_in_genre(genre, genre_dict, avg_ratings, n=5):
    movies_in_genre = genre_dict.get(genre, [])
    genre_avg_ratings = avg_ratings[movies_in_genre]
    sorted_genre_movies = genre_avg_ratings.sort_values(ascending=False)
    return sorted_genre_movies.head(n)

Series([], dtype: float64)


In [99]:
pop_in_genre = get_popular_in_genre("Adventure", genre_dict, avg_ratings, 2)
assert list(pop_in_genre.index) == [1, 2]
assert pop_in_genre.is_monotonic_decreasing

AssertionError: 

#### 3.4 (8 points) Genre Ratings

One important analysis for the content platforms is to determine
ratings by genre.

Write a function `get_genre_rating` that takes the same parameters as
`get_popular_in_genre` above, except for $n$, and returns the average
rating of the movies in the given genre.
```
[In]: get_genre_rating("Action", genre_dict, avg_ratings)
[Out]: 3.527777777777778
```

In [58]:
import pandas as pd

def get_genre_rating(genre, genre_dict, avg_ratings):
    # Check if the specified genre is in the genre dictionary
    if genre in genre_dict:
        movie_ids = genre_dict[genre]
    
        genre_movies = avg_ratings[avg_ratings.index.isin(movie_ids)]
        
        average_genre_rating = genre_movies.mean()
        
        return average_genre_rating
    
    return -1

# Example usage
genre_rating = get_genre_rating("Action", genre_dict, avg_ratings)

# Check if the genre rating is within the expected range
assert genre_rating > 3.525
assert genre_rating < 3.53


AssertionError: 

In [59]:
genre_rating = get_genre_rating("Action", genre_dict, avg_ratings)
assert genre_rating > 3.525
assert genre_rating < 3.53

AssertionError: 

#### 3.5 (8 points) Movie of the Year

Write a function `get_movie_of_the_year` which takes as parameters a year, a movie-to-average rating Series (as created in 2.2), and a movies DataFrame (as created in 1.1)
and returns the title of the movie with the highest average rating 
for the given year. 
```
[In]: get_movie_of_the_year(1995, avg_ratings, movies_df)
[Out]: 'Heat'
```

In [61]:
import pandas as pd

def get_movie_of_the_year(year, avg_ratings, movies_df):
    movies_for_year = movies_df[movies_df['year'] == year]

    if not movies_for_year.empty:
        movie_ids = movies_for_year.index
        year_movies_ratings = avg_ratings[avg_ratings.index.isin(movie_ids)]

        best_movies_ids = year_movies_ratings[year_movies_ratings == year_movies_ratings.max()].index


        best_movies_titles = [movies_df.loc[movie_id, 'title'] for movie_id in best_movies_ids]

        return best_movies_titles


    return None



AssertionError: 

In [62]:
assert get_movie_of_the_year(1995, avg_ratings, movies_df) == 'Heat'

AssertionError: 

### Part 4: User Focused

In [ ]:
from hw3 import read_user_ratings, get_user_genre, recommend_movies

#### 4.1 **(8 points) User Ratings**

Read the ratings file to return a user-to-movies dictionary that maps user ID to the associated movies
and the corresponding ratings. Write a function named `read_user_ratings` for this, with the ratings
file as the parameter.

For example:
```
{
    u1: [(m1, r1), (m2, r2)],
    u2: [(m3, r3), (m8, r8)]
}
```
where `u1` is a user ID, `m1` is a movie ID, and `r1` is the corresponding rating. 



In [1]:
import pandas as pd

def read_user_ratings(ratings_file):
    user_movies = {}
    with open(ratings_file, 'r') as file:
        for line in file:
            user_id, movie_id, rating = map(int, line.strip().split(','))
            if user_id not in user_movies:
                user_movies[user_id] = []
            user_movies[user_id].append((movie_id, rating))
    return user_movies

In [2]:
assert user_movies[42] == [(3, 4.0)]

NameError: name 'user_movies' is not defined

#### 4.2 (10 points) User Genre

Write a function `get_user_genre` that takes as parameters a user id, the user-to-movies dictionary
(as created in Part 4.1 above), and the movies DataFrame (as created in Part 1.1), and returns
the top genre that the user likes based on the user’s ratings. Here, the top genre for the user will be
determined by taking the average rating of the movies genre-wise that the user has rated. If multiple
genres have the same highest ratings for the user, return any one of genres (arbitrarily) as the top
genre.

```
[In]: get_user_genre(6, user_movies, movies_df)
[Out]: 'Comedy'
```

In [3]:
import pandas as pd

def get_user_genre(user_id, user_movies, movies_df):
    genre_ratings = {}
    for movie_id, rating in user_movies[user_id]:
        genre = movies_df[movies_df['movieId'] == movie_id]['genre'].values[0]
        if genre not in genre_ratings:
            genre_ratings[genre] = []
        genre_ratings[genre].append(rating)

    top_genre = max(genre_ratings, key=lambda genre: sum(genre_ratings[genre]) / len(genre_ratings[genre]))
    return top_genre

In [4]:
user_genre = get_user_genre(42, user_movies, movies_df)
assert user_genre == "Comedy"

NameError: name 'user_movies' is not defined

#### 4.3 (10 points) User Recommendations

Recommend 3 most popular (highest average rating) movies from the user’s top genre that the user
has not yet rated. Write a function called `recommend_movies` for this, that takes as parameters a user id,
the user-to-movies dictionary (as created in Part 4.1 above), the movies DataFrame (as created
in Part 1.1), and the movie-to-average rating Series (as created in Part 2.2). The function should
return a pandas Series of movie-to-average rating, sorted in decreasing order by rating. If fewer than 3 movies make the cut, then return all
the movies that make the cut in order of top average ratings.
```
[In]: recommend_movies(6, user_movies, movies_df, avg_ratings)
[Out]: 
3    4.0
5    4.0
4    2.5
Name: avg_ratings, dtype: float64
```

In [5]:
import pandas as pd
def recommend_movies(user_id, user_movies, movies_df, avg_ratings):
    user_genre = get_user_genre(user_id, user_movies, movies_df)
    unrated_movies = movies_df[~movies_df['movieId'].isin([movie_id for movie_id, _ in user_movies[user_id]])]
    unrated_movies = unrated_movies[unrated_movies['genre'] == user_genre]
    unrated_movies = unrated_movies.merge(avg_ratings, on='movieId', how='left')

    recommended_movies = unrated_movies.sort_values(by='avg_ratings', ascending=False)
    return recommended_movies[['movieId', 'avg_ratings']].head(3)